In [1]:
import pandas as pd
import pymysql
import logging
import sshtunnel
from creds import Creds
from sshtunnel import SSHTunnelForwarder

In [17]:
ssh_host = Creds.ssh_host
ssh_username = Creds.ssh_username
ssh_password = Creds.ssh_password
database_username = Creds.database_username
database_password = Creds.database_password
database_name = Creds.database_name
localhost = Creds.localhost

In [18]:
def open_ssh_tunnel(verbose=False):
    """Open an SSH tunnel and connect using a username and password.

    :param verbose: Set to True to show logging
    :return tunnel: Global SSH tunnel connection
    """

    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG

    global tunnel
    tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_password = ssh_password,
        remote_bind_address = ('127.0.0.1', 3306)
    )

    tunnel.start()

In [19]:
def mysql_connect():
    """Connect to a MySQL server using the SSH tunnel connection

    :return connection: Global MySQL database connection
    """

    global connection

    connection = pymysql.connect(
        host=localhost,
        user=database_username,
        passwd=database_password,
        db=database_name,
        port=tunnel.local_bind_port
    )

In [20]:
def run_query(sql):
    """Runs a given SQL query via the global database connection.

    :param sql: MySQL query
    :return: Pandas dataframe containing results
    """

    return pd.read_sql_query(sql, connection)

In [21]:
open_ssh_tunnel()
mysql_connect()
df = run_query('SELECT * FROM deals LIMIT 100')
df.head()

OperationalError: (1045, "Access denied for user 'mityashekhov'@'localhost' (using password: YES)")